In [1]:
from string import ascii_lowercase
import random
from itertools import combinations
import numpy as np

In [2]:
def createproblem(n,k,m):
        pos_var = list(ascii_lowercase)[:n]
        neg_var = [var.upper() for var in pos_var]
        variables = pos_var + neg_var
        problem = []
        threshold = 10       
        i = 0
        combination = list(combinations(variables,k))
        
        while i<threshold:
            c = random.sample(combination,m)
            if c not in problem:
                problem.append(c)
                i += 1
        
        new_problem = []
        for c in problem:
            temp = []
            temp = [list(sub) for sub in c]
            new_problem.append(temp)
        return  variables,new_problem 
     
    
 
        

In [3]:
def random_assign(variables,n):
    literal = list(np.random.choice(2,n))
    negation = [abs(i-1) for i in literal]
    assign = literal + negation
    return dict(zip(variables,assign))

def heuristic(problem,assign):
    count = 0
    for sub in problem:       
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count    
    
    

In [4]:
def next_node(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for k in key:
        temp = current.copy()
        temp[k] = abs(temp[k]-1)
        temp[chr(ord(k)+32)] = abs(temp[chr(ord(k)+32)]-1)
        successors.append(temp)  
    

    
    return successors
    

In [5]:
print(next_node({'a': 0, 'b': 0, 'c': 0, 'A': 1, 'B': 1, 'C': 1}))

[{'a': 1, 'b': 0, 'c': 0, 'A': 0, 'B': 1, 'C': 1}, {'a': 0, 'b': 1, 'c': 0, 'A': 1, 'B': 0, 'C': 1}, {'a': 0, 'b': 0, 'c': 1, 'A': 1, 'B': 1, 'C': 0}]


In [8]:
def select_node(successor,problem):
    heuristic_val = []
    for i in successor:
        heuristic_val.append(heuristic(problem,i))
    index = heuristic_val.index(max(heuristic_val))
    return successor[index]

def check_goal_state(state,problem):
    count = 0
    for sub in problem:       
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

def hill_climbing(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        print(current_state,step)
        return 
    else:
        step += 1
        successor = next_node(current_state)
        new_node = select_node(successor,problem)
        hill_climbing(new_node,problem,step)

In [10]:
n = 2
k = 3
m = 2
var,prob = createproblem(n,k,m)
print(var)
for i in prob:
    print(i)
    

['a', 'b', 'A', 'B']
[['a', 'b', 'A'], ['b', 'A', 'B']]
[['b', 'A', 'B'], ['a', 'b', 'A']]
[['a', 'b', 'A'], ['a', 'b', 'B']]
[['b', 'A', 'B'], ['a', 'b', 'B']]
[['a', 'A', 'B'], ['b', 'A', 'B']]
[['a', 'b', 'A'], ['a', 'A', 'B']]
[['a', 'b', 'B'], ['a', 'b', 'A']]
[['a', 'A', 'B'], ['a', 'b', 'A']]
[['a', 'b', 'B'], ['a', 'A', 'B']]
[['a', 'b', 'B'], ['b', 'A', 'B']]


In [11]:
start_state = random_assign(var,n)
print(start_state,prob[0])
# start_state = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f':0,'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'F':1}
hill_climbing(start_state,prob[0],0)

{'a': 1, 'b': 1, 'A': 0, 'B': 0} [['a', 'b', 'A'], ['b', 'A', 'B']]
{'a': 1, 'b': 1, 'A': 0, 'B': 0} 0


## BEAM SEARCH

In [12]:
def heuristic(problem,assign):
    count = 0
    for sub in problem:       
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count
    
def select_node_beam(successor,problem,beamwidth):
    heuristic_val = []
    beam_nodes = []
    for i in successor:
        heuristic_val.append(heuristic(problem,i))
    for i in range(beamwidth):
        index = heuristic_val.index(max(heuristic_val))
        beam_nodes.append(successor[index])
        successor.remove(successor[index])
        
    return beam_nodes


In [13]:
def check_goal_state(state,problem,beamwidth):
    count = 0
    for sub in problem:       
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count


    

In [14]:
def next_node_beam(succs):
    nextnode= []
    for current in succs:
        key = list(current.keys())
        key.sort()
        key = key[0:len(key)//2]
        successors = []
        for k in key:
            temp = current.copy()
            temp[k] = abs(temp[k]-1)
            temp[chr(ord(k)+32)] = abs(temp[chr(ord(k)+32)]-1)
            successors.append(temp)
            nextnode.append(temp)
        
    return nextnode



In [15]:

def beam_search(start,problem,step,beamwidth):
    if step == 10000:
        print('halted at depth',step)
        return
    for current_state in start:
        print(current_state)
        if check_goal_state(current_state,problem)==True:
            print(current_state,step)
            return 
    step +=1    
    successor = next_node_beam(start,problem,beamwidth)
    new_node = select_node_beam(successor,prob[0],beamwidth)
    beam_search(start,problem,step,beamwidth)

In [16]:
start_state = random_assign(var,n)
print([start_state,start_state])

[{'a': 1, 'b': 0, 'A': 0, 'B': 1}, {'a': 1, 'b': 0, 'A': 0, 'B': 1}]


# Variable-Neighborhood-Descent with 3 neighborhood functions


In [17]:
def heuristic(problem,assign):
    count = 0
    for sub in problem:       
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count


In [18]:
def check_goal_state(state,problem):
    count = 0
    for sub in problem:       
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

In [19]:
def select_node(successor,problem):
    heuristic_val = []
    for i in successor:
        heuristic_val.append(heuristic(problem,i))
    index = heuristic_val.index(max(heuristic_val))
    return successor[index]

In [20]:
def nghd1(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for k in key:
        temp = current.copy()
        temp[k] = abs(temp[k]-1)
        temp[chr(ord(k)+32)] = abs(temp[chr(ord(k)+32)]-1)
        successors.append(temp)   

    
    return successors

In [21]:
print(nghd1({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))


[{'a': 0, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 0, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}]


In [22]:
def nghd2(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for j in range(0,len(key)-1):
        for i in range(j,len(key)-1):
            
            temp = current.copy()
            temp[key[i]] = abs(temp[key[i]]-1)
            temp[key[i+1]] = abs(temp[key[i+1]]-1)
            temp[chr(ord(key[i])+32)] = abs(temp[chr(ord(key[i])+32)]-1)
            temp[chr(ord(key[i+1])+32)] = abs(temp[chr(ord(key[i+1])+32)]-1)
            successors.append(temp)   

    
    return successors

In [23]:
print(nghd2({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))


[{'a': 0, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}]


In [24]:
def nghd3(current):
    key = list(current.keys())
    key.sort()
    key = key[0:len(key)//2]
    successors = []
    for j in range(0,len(key)-2):
        for k in range(j,len(key)-2):
            for i in range(k,len(key)-2):               
            
                temp = current.copy()
                temp[key[i]] = abs(temp[key[i]]-1)
                temp[key[i+1]] = abs(temp[key[i+1]]-1)
                temp[chr(ord(key[i])+32)] = abs(temp[chr(ord(key[i])+32)]-1)
                temp[chr(ord(key[i+1])+32)] = abs(temp[chr(ord(key[i+1])+32)]-1)
                successors.append(temp)   

    
    return successors

In [25]:
print(nghd3({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))


[{'a': 0, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}]


In [27]:
def Variable_Neighborhood1(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        return current_state
        
    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd1(successor,problem)
        Variable_Neighborhood(new_node,problem,step)

In [28]:
def Variable_Neighborhood2(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        return current_state
         
    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd2(successor,problem)
        Variable_Neighborhood(new_node,problem,step)

In [29]:
def Variable_Neighborhood3(current_state,problem,step):
    if step == 10000:
        print('halted at depth',step)
        return
    if check_goal_state(current_state,problem)==True:
        print(current_state,step)
        return 
    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd3(successor,problem)
        Variable_Neighborhood(new_node,problem,step)

In [30]:
n = 5
k = 3
m = 5
var,probl = createproblem(n,k,m)
print(var)
for i in probl:
    print(i)

['a', 'b', 'c', 'd', 'e', 'A', 'B', 'C', 'D', 'E']
[['e', 'A', 'D'], ['e', 'B', 'D'], ['d', 'e', 'D'], ['c', 'd', 'A'], ['a', 'e', 'D']]
[['a', 'b', 'd'], ['b', 'c', 'E'], ['c', 'e', 'A'], ['b', 'C', 'D'], ['d', 'B', 'D']]
[['c', 'd', 'C'], ['b', 'd', 'E'], ['a', 'b', 'C'], ['b', 'A', 'D'], ['a', 'B', 'E']]
[['c', 'A', 'D'], ['d', 'C', 'E'], ['d', 'B', 'E'], ['b', 'd', 'e'], ['a', 'e', 'C']]
[['b', 'e', 'B'], ['d', 'A', 'D'], ['a', 'c', 'B'], ['c', 'e', 'E'], ['d', 'A', 'B']]
[['c', 'e', 'B'], ['b', 'B', 'D'], ['c', 'e', 'A'], ['c', 'e', 'E'], ['A', 'D', 'E']]
[['a', 'B', 'E'], ['a', 'C', 'D'], ['b', 'c', 'e'], ['a', 'e', 'D'], ['A', 'C', 'D']]
[['d', 'A', 'E'], ['b', 'e', 'B'], ['a', 'e', 'A'], ['d', 'B', 'C'], ['b', 'd', 'D']]
[['d', 'C', 'E'], ['e', 'A', 'E'], ['e', 'D', 'E'], ['A', 'D', 'E'], ['a', 'b', 'E']]
[['b', 'C', 'D'], ['d', 'B', 'E'], ['a', 'b', 'E'], ['a', 'D', 'E'], ['a', 'C', 'D']]


In [31]:
start_state = random_assign(var,n)
print([start_state,start_state])


[{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1}, {'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1}]


In [32]:
var1 = start_state
var2 = Variable_Neighborhood1(var1,prob[0],0)
var3 = Variable_Neighborhood2(var2,prob[0],0)
Variable_Neighborhood1(var3,prob[0],0)

{'a': 0,
 'b': 1,
 'c': 0,
 'd': 0,
 'e': 0,
 'A': 1,
 'B': 0,
 'C': 1,
 'D': 1,
 'E': 1}